# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load the csv file
df = pd.read_csv('../my_output_data/city_info.csv')
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date (utc)
0,Zonguldak,41.25,31.83,59.49,52,0,4.27,TR,2020-10-13
1,Mossamedes,-15.20,12.15,61.11,86,0,2.64,AO,2020-10-13
2,Huarmey,-10.07,-78.15,62.80,84,13,6.73,PE,2020-10-13
3,Leningradskiy,69.38,178.42,23.20,88,90,3.06,RU,2020-10-13
4,Severo-Kuril'sk,50.68,156.12,48.94,74,99,20.22,RU,2020-10-13
...,...,...,...,...,...,...,...,...,...
541,Atuona,-9.80,-139.03,78.66,71,41,13.96,PF,2020-10-13
542,Camacupa,-12.02,17.48,64.38,75,100,4.36,AO,2020-10-13
543,Ilulissat,69.22,-51.10,32.00,92,100,4.79,GL,2020-10-13
544,Hobart,-42.88,147.33,60.01,43,20,13.87,AU,2020-10-13


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# enable the display gmap
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
#set up the figure layout
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
#create the empyty list to store the data generated
coordinates=[]
hums = []
# for loop to convert the df to the list
for index, row in df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    hum = row['Humidity']
    coordinate = (lat,lng)
    coordinates.append(coordinate)
    hums.append(hum)
#load the figure_layout
fig = gmaps.figure(layout=figure_layout)
#set up the heatmap layer based on the coordinates and humunity
heatmap = gmaps.heatmap_layer(coordinates,weights=hums,dissipating=True,max_intensity = df['Humidity'].max())
#add the heatlayer
fig.add_layer(heatmap)
#show the fig
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# set up the filt for the selected cities
filt = (df['Max Temp']>70)&(df['Max Temp']<90)&(df['Cloudiness']==0)&(df['Wind Speed']<10)
# use loc to filt the original df and generate the selected_df
selected_df = df.loc[filt]
selected_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date (utc)
18,Mānsa,29.98,75.38,72.41,30,0,4.61,IN,2020-10-13
33,Tura,25.52,90.22,79.75,78,0,2.13,IN,2020-10-13
89,Sharjah city,25.36,55.40,84.20,39,0,8.05,AE,2020-10-13
109,Niterói,-22.88,-43.10,73.40,73,0,4.70,BR,2020-10-13
167,Carutapera,-1.20,-46.02,77.18,85,0,6.96,BR,2020-10-13
177,Ambilobe,-13.20,49.05,72.90,81,0,4.36,MG,2020-10-13
228,Tura,25.52,90.22,79.75,78,0,2.13,IN,2020-10-13
266,Porto Walter,-8.27,-72.74,77.41,67,0,3.15,BR,2020-10-13
270,Makkah al Mukarramah,21.43,39.83,79.16,29,0,8.08,SA,2020-10-13
290,Mau,25.95,83.55,80.42,73,0,5.59,IN,2020-10-13


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Store into variable named hotel_df.
hotel_df=pd.DataFrame()
#set up the parameters for nearbysearch
# target_search = "hotel"
target_radius = 5000
target_type = "lodging"
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#set up params
params = {
#     "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
# set up the empyty list
hotel_names=[]
city_names=[]
country_names=[]
lats=[]
lngs=[]
#for loop to get the row data
for index, row in selected_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    coordinate=(lat,lng)
    # try to ger the city info
    try:
        places_data = requests.get(base_url, params=params).json()
        hotel_name=places_data['results'][0]["name"]
        hotel_names.append(hotel_name)
        city_names.append(row['City'])
        country_names.append(row['Country'])
        coordinates.append(coordinate)
        lats.append(lat)
        lngs.append(lng)
    except (IndexError):
        pass
# load the collected data to the df
hotel_df['Hotel Name']=hotel_names
hotel_df['City']=city_names
hotel_df['Country']=country_names
hotel_df['Lat']=lats
hotel_df['Lng']=lngs
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,STARiHOTEL Mansa,Mānsa,IN,29.98,75.38
1,Hotel RIKMAN Continental,Tura,IN,25.52,90.22
2,Swiss Belhotel Sharjah,Sharjah city,AE,25.36,55.40
3,OYO Viareggio Hotel - Niterói,Niterói,BR,-22.88,-43.10
4,Lidera,Carutapera,BR,-1.20,-46.02
5,Hôtel & Restaurant DIANA Ambilobe,Ambilobe,MG,-13.20,49.05
6,Hotel RIKMAN Continental,Tura,IN,25.52,90.22
7,Hotel Silva,Porto Walter,BR,-8.27,-72.74
8,Hotel Fairmont Makkah Clock Royal Tower,Makkah al Mukarramah,SA,21.43,39.83
9,Dr Sajid Mahmood,Mau,IN,25.95,83.55


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(marker_layer)
fig.add_layer(heatmap)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…